# Lab-10-4 ImageFolder

[1]

## 지난시간에는
- Visdom 사용법 및 적용

## 오늘은!
- 내가 가진 사진을 이용해서 딥러닝 모델 학습해보기

## 진행순서
1. 나만의 데이터 셋 준비하기
2. torchvision.datasets.ImageFolder으로 불러오기
3. transforms 적용하여 저장 하기 origin_data -> train_data

## 나만의 데이터 셋 준비하기
- 저는 connect 재단에 있던 의자를 딥러닝 모델로 구분해보려고 해요
- 여러분도 여러분의 사진을 준비해보세요!
- 연습이니까 **시각적으로 명확하게 구분되는 사진**을 사용하시는 걸 **추천** 드립니다.

![snapshot](./10-4-1-0.png)

![snapshot](./10-4-1-1.png)

![snapshot](./10-4-1-2.png)

## 직접해보기
- jupyter notebook으로~

In [1]:
import torchvision
from torchvision import transforms

from torch.utils.data import DataLoader

In [2]:
from matplotlib.pyplot import imshow

# matplotlib의 출력값이 주피터 노트북 상에 표현되게 해주세요. 인라인으로 해달라. 이런 말.
%matplotlib inline

In [7]:
# 먼저 수행하기 전에, 내가 있는 위치의 어디에 데이터셋이 있는지를 확인해봐야 함.
# ![COMMAND] : e.g., !ls custom_data/origin_data -> "gray red" (folders)
!ls

01-1-1_numpy.ipynb    07-2.ipynb            10-2-0.png
01-1-2_torch.ipynb    08-1-perceptron.ipynb 10-2-1.png
01-2.ipynb            08-2-mlp.ipynb        10-2.ipynb
02.ipynb              09-1.ipynb            10-3.ipynb
03.ipynb              09-2.ipynb            10-4-1-0.png
04-1.ipynb            09-3.ipynb            10-4-1-1.png
04-2.ipynb            09-4.ipynb            10-4-1-2.png
05.ipynb              10-1-0.png            10-4-1.ipynb
06.ipynb              10-1-1.png            MNIST_data
07-1.ipynb            10-1.ipynb            cifar10


In [ ]:
#train_data = torchvision.datasets.ImageFolder(root="custom_data/origin_data", transform=None)

# transform을 많이 수행해야 되면 그걸 일일이 여기에다 넣어주는 게 귀찮다. 그래서 그걸 하나로 묶어주는 역할.
trans = transforms.Compose([
    transforms.Resize((64, 128))
])
train_data = torchvision.datasets.ImageFolder(root="custom_data/origin_data", transform=trans)

In [ ]:
for num, value in enumerate(train_data):
    data, label = value
    print(num, data, label)
    
    # trans한 data를 다시 저장
    if label == 0:  # 0(gray) 아니면 1(red) 밖에 라벨이 없기 때문에 이렇게 진행하겠다.
        data.save('custom_data/train_data/gray/%d_%d.jpeg' % (num, label))
    else:
        data.save('custom_data/train_data/red/%d_%d.jpeg' % (num, label))
    
    '''
    imshow(data)   # image 출력
    break
    '''

![snapshot](./10-4-1-3.png)

## 오늘 같이 해본 것들은?
- 내가 가진 사진을 이용해서 dataset을 만드는 법

## What's Next?
- CNN training

[2]

## 지난시간에는
- ImageFolder 사용법

## 오늘은
- 내 사진을 분류하는 딥러닝 모델 학습

![snapshot](./10-4-2-0.png)

*[16, 13, 29] : (C, H, W)*

## 오늘 같이 해볼 것들은?
- 내가 가진 사진을 이용해서 dataset을 만드는 법
- 배운 내용을 이용해서 학습하기
- 모델을 저장하고 다시 불러오는 법

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import torch.optim as optim
from torch.utils.data import DataLoader

import torchvision
import torchvision.transforms as transforms



device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)



trans = transforms.Compose([
    transforms.ToTensor()
])

train_data = torchvision.datasets.ImageFolder(root='./custom_data/train_data', transform=trans)



data_loader = DataLoader(dataset=train_data, batch_size=8, shuffle=True, num_workers=2)



class CNN(nn.Module):
    
    def __init__(self):
        super(CNN, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(3, 6, 5),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        self.layer2 = nn.Sequential(
            nn.Conv2d(6, 16, 5),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        self.layer3 = nn.Sequential(
            nn.Conv2d(16*13*29, 120),
            nn.ReLU(),
            nn.Linear(120,2)
        )
        
    def forward(self, x):
        # 일반적으로 레이어 사이즈 바꾸게 되면 이 부분에서 귀찮게 되는 경우가 있음.
        # 편하게 할 수 있는 방법은 아래처럼 shape 넣어서 테스트하는 게 가장 좋은 방법이다.
        
        out = self.layer1(x)
        #print(out.shape) # 이렇게 하면 각 레이어 통과해서 크기 어떻게 되는지 바로바로 잡혀서 좋다.
        out = self.layer2(out)
        #print(out.shape)
        out = out.view(out.shape[0], -1)
        #print(out.shape)
        out = self.layer3(out)
        return out
    
    

#testing
net = CNN().to(device)
test_input = (torch.Tensor(3,3,64,128)).to(device)
test_out = net(test_input)



optimizer = optim.Adam(net.parameters(), lr=0.00001)
loss_func = nn.CrossEntropyLoss().to(device)



total_batch = len(data_loader)

epochs = 5   # 데이터셋이 워낙 쉽기 때문에 잘 학습이 되니까 3번만 돌리자.
for epoch in range(epochs):
    avg_cost = 0.0
    
    for num, data in enumerate(data_loader):
        imgs, labels = data
        imgs = imgs.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        out = net(imgs)
        loss = loss_func(out, labels)
        loss.backward()
        optimizer.step()
        
        avg_cost += loss / total_batch
    
    print('[Epoth:{}] cost = {}'.format(epoch+1, avg_cost))
print('Learning Finished!')

In [ ]:
# 모델을 save해서 다시 불러오는 방법
torch.save(net.state_dict(), "./model/model.pth")

In [ ]:
new_net = CNN().to(device)
new_net.load_state_dict(torch.load('./model/model.pth'))

In [ ]:
# 불러온 게 같은지 확인 한번 해본 것

print(net.layer1[0])
print(new_net.layer1[0])

print(net.layer1[0].weight[0][0][0])
print(new_net.layer1[0].weight[0][0][0])

net.layer1[0].weight[0] == new_net.layer1[0].weight[0]

In [ ]:
trans = torchvision.transforms.Compose([
    transforms.Resize((64, 128)),
    transforms.ToTensor()
])
test_data = torchvision.datasets.ImageFolder(root='./custom_data/test_data', transform=trans)

In [ ]:
test_set = DataLoader(dataset=test_data, batch_size=len(test_data))

In [ ]:
with torch.no_grad():
    for num, data in enumerate(test_set):
        imgs, label = data
        imgs = imgs.to(device)
        label = label.to(device)
        
        prediction = net(imgs)
        
        correct_prediction = torch.argmax(prediction, 1) == label
        
        accuracy = correct_prediction.float().mean()
        print('Accuracy:', accuracy.item())

## What's Next?
- Advanced CNN (AlexNet, VGGNet, ResNet, DenseNet, SENet, AutoML, ...)
    - 하나하나 다 설명하긴 힘들고, 이전에 올린 강의에 이론 있고, 여기서는 실습 위주로 많이 다뤄보려고 함.